In [1]:
import pandas as pd
import numpy as np
import os
from decimal import Decimal
import pickle
from pathlib import Path

# SETUP

In [2]:
dir_tree_util_path = os.path.join("utils", "dir_tree.py")
exec(open(dir_tree_util_path).read())

# INPUTS

In [3]:
# Paths
path_dados_pib = PROJECT_DIRS['DADOS_GIT_BACEN_DIR']
# path_output = TODO
path_output = PROJECT_DIRS['DADOS_DERIVADOS_DIR']

# CARREGANDO p/MEMORIA OS DADOS

In [4]:
# df_pib mensal nominal
# fonte: Bacen. SGS, df_pib mensal - valores correntes, Cód 4380
df_pib = pd.read_csv(path_dados_pib / 'STP-20240827171150836.csv', sep=";", encoding='windows-1251', 
                  skipfooter=1, engine='python')

In [5]:
# df_ipca mensal
# fonte: Bacen. SGS, Índice nacional de preços ao consumidor-amplo (df_ipca), Cód 433
df_ipca = pd.read_csv(path_dados_pib / 'STP-20240827171950785.csv', sep=";", encoding='windows-1251', 
                  skipfooter=1, engine='python')

# TRATANDO OS DADOS

## IPCA

In [6]:
df_ipca

,Data,433 - Нndice nacional de preзos ao consumidor-amplo (IPCA) - Var. % mensal
0,01/1980,"6,62"
1,02/1980,"4,62"
2,03/1980,"6,04"
3,04/1980,"5,29"
4,05/1980,"5,70"
...,...,...
530,03/2024,"0,16"
531,04/2024,"0,38"
532,05/2024,"0,46"
533,06/2024,"0,21"


In [7]:
df_ipca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535 entries, 0 to 534
Data columns (total 2 columns):
 #   Column                                                                      Non-Null Count  Dtype 
---  ------                                                                      --------------  ----- 
 0   Data                                                                        535 non-null    object
 1   433 - Нndice nacional de preзos ao consumidor-amplo (IPCA) - Var. % mensal  535 non-null    object
dtypes: object(2)
memory usage: 8.5+ KB


In [8]:
df_ipca.loc[0,'Data']

'01/1980'

In [9]:
type(df_ipca.loc[0,'Data'])

str

In [10]:
# Simplificando nome de coluna
df_ipca = df_ipca.rename(columns={'433 - Нndice nacional de preзos ao consumidor-amplo (IPCA) - Var. % mensal':'ipca_mensal_%'})

In [11]:
# Convertendo coluna de data para tipo datetime, e p/final do mês
df_ipca['Data'] = pd.to_datetime(df_ipca['Data'], format='%m/%Y')
df_ipca['Data'] = df_ipca['Data'].dt.to_period('M').dt.to_timestamp('M')

In [12]:
# Convertendo ipca de string para float
type(df_ipca.loc[0,'ipca_mensal_%']) # str
df_ipca['ipca_mensal_%'] = df_ipca['ipca_mensal_%'].str.replace(',','.')
df_ipca['ipca_mensal_%'] = df_ipca['ipca_mensal_%'].astype('float')

In [13]:
df_ipca['Data'].dtype

dtype('<M8[ns]')

In [14]:
# Atribuindo dez/2009 = 1.00
df_ipca['ipca_mensal'] = 1 + df_ipca['ipca_mensal_%']/100
df_ipca['ipca_acum'] =  df_ipca['ipca_mensal'].cumprod()
scaling_date = df_ipca.loc[df_ipca['Data'] == '2009-12-31']['ipca_acum'].iloc[0]
df_ipca['ipca_acum'] = df_ipca['ipca_acum'] / scaling_date

## PIB

In [15]:
df_pib

,Data,4380 - PIB mensal - Valores correntes (R$ milhхes) - R$ (milhхes)
0,01/1990,"0,2"
1,02/1990,"0,4"
2,03/1990,"0,8"
3,04/1990,"0,7"
4,05/1990,"0,8"
...,...,...
410,03/2024,"952.242,4"
411,04/2024,"977.372,0"
412,05/2024,"954.779,1"
413,06/2024,"952.995,6"


In [16]:
df_pib.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 2 columns):
 #   Column                                                             Non-Null Count  Dtype 
---  ------                                                             --------------  ----- 
 0   Data                                                               415 non-null    object
 1   4380 - PIB mensal - Valores correntes (R$ milhхes) - R$ (milhхes)  415 non-null    object
dtypes: object(2)
memory usage: 6.6+ KB


In [17]:
df_pib = df_pib.rename(columns={
'4380 - PIB mensal - Valores correntes (R$ milhхes) - R$ (milhхes)':'PIB_mensal'
})

In [18]:
# Alterando data para datetime e final do mês
df_pib['Data'] = pd.to_datetime(df_pib['Data'], format='%m/%Y')
df_pib['Data'] = df_pib['Data'].dt.to_period('M').dt.to_timestamp('M')
# df_pib['Data'] = df_pib['Data'] + pd.offsets.MonthEnd(0)

In [19]:
# Alterando PIB para float
df_pib['PIB_mensal'] = df_pib['PIB_mensal'].str.replace('.', '', regex=False)
df_pib['PIB_mensal'] = df_pib['PIB_mensal'].str.replace(',', '.', regex=False)
df_pib['PIB_mensal'] = df_pib['PIB_mensal'].astype('float')

# Calculando o PIB real

In [20]:
df_pib = df_pib.merge(df_ipca[['Data','ipca_acum']], on='Data')

In [21]:
df_pib['PIB_real'] = df_pib['PIB_mensal'] / df_pib['ipca_acum']

# Calculando a variação do PIB mensal

In [22]:
df_pib['PIB_mensal_real_%'] = df_pib['PIB_real'].pct_change() * 100

# Descartando colunas que não serão utilizadas

In [23]:
df_pib = df_pib.drop(columns={'PIB_mensal','ipca_acum','PIB_real'})

In [24]:
df_pib['Data'] = df_pib['Data'].dt.date

# SALVANDO PARA PARQUET

In [25]:
df_pib.to_parquet(path_output / 'PIB-Bacen_mensal.parquet')